Data:

https://www.kaggle.com/code/roydatascience/us-election-results-2020/input?select=president_county_candidate.csv

# Note:

Maps are interactive (zoom and hover for numerical data) when jupyter files ran, but GitHub does not support this feature, so png versions of maps were added. 

In [1]:
import numpy as np 
import pandas as pd 
import plotly.figure_factory as ff
import plotly.express as px
import time
from datetime import datetime
import kaleido

data = pd.read_csv('/Users/kellyg/maps-project/data/president_state.csv')
state_codes = pd.read_csv('/Users/kellyg/maps-project/data/world_country_and_usa_states_latitude_and_longitude_values.csv')
datafile = data.merge(state_codes, left_on='state', right_on='usa_state')

fig = px.choropleth(datafile, 
                    locations='usa_state_code', 
                    color="total_votes",
                    color_continuous_scale="PuOr", 
                    range_color=(0, 10000000),
                    locationmode = 'USA-states',  
                    scope="usa",
                    title='USA Presidential Votes Counts' 
                    )
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})

# fig.write_image("votes.png")

fig.show()


![Figure](map_png/votes.png)

In [2]:
import pandas as pd
import plotly.express as px

data = pd.read_csv('/Users/kellyg/maps-project/data/president_state.csv')
state_codes = pd.read_csv('/Users/kellyg/maps-project/data/world_country_and_usa_states_latitude_and_longitude_values.csv')
pop = pd.read_csv('/Users/kellyg/maps-project/data/states_pop.csv')
datafile = data.merge(state_codes, left_on='state', right_on='usa_state')

votes_percapita = datafile.merge(pop, on='usa_state_code')

#votes per capita
votes_percapita['votes_per_capita'] = votes_percapita['total_votes'] / votes_percapita['population']

fig = px.choropleth(votes_percapita, 
                    locations='usa_state_code', 
                    color="votes_per_capita", 
                    color_continuous_scale="PuOr", 
                    locationmode='USA-states',  
                    scope="usa",
                    title='USA Presidential Votes Per Capita' 
                    )
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
# fig.write_image("vote_capita.png")

fig.show()


![Figure](map_png/vote_capita.png)

# Votes by Party
Using 2020 election data, we can easily visualize which states were Democrat/Republican, especially using a red/blue color scale. Some of this code for this first map was taken from the Kaggle notebook to practice interacting with the dataset.

In [3]:

import warnings
warnings.filterwarnings("ignore")

df_president_county = pd.read_csv('/Users/kellyg/maps-project/data/president_county_candidate.csv')
df_president_county = df_president_county[df_president_county.party.isin(['DEM','REP'])]
df_president_county = df_president_county.groupby(['state','party'])["total_votes"].sum()
df_president_county = df_president_county.reset_index()

state_codes = pd.read_csv('/Users/kellyg/maps-project/data/world_country_and_usa_states_latitude_and_longitude_values.csv')
state_codes = state_codes[['usa_state','usa_state_code']]
df_president_county = df_president_county.merge(state_codes, left_on='state', right_on='usa_state')
df_president_county = df_president_county.drop(['usa_state'], axis=1)
df_president_county_dummy = pd.get_dummies(df_president_county['party'])

df_president_county = df_president_county.join(df_president_county_dummy)
df_president_county['DEM_votes'] = df_president_county['DEM'] * df_president_county['total_votes'] 
df_president_county['REP_votes'] = df_president_county['REP'] * df_president_county['total_votes'] 
df_president_county = df_president_county.groupby(['state', 'usa_state_code'])[["DEM_votes", "REP_votes"]].sum()
df_president_county = df_president_county.reset_index()
df_president_county['percent_democrat'] = df_president_county['DEM_votes']*100/(df_president_county['REP_votes']+df_president_county['DEM_votes'])

fig = px.choropleth(df_president_county, 
                    locations="usa_state_code", 
                    color = "percent_democrat",
                    locationmode = 'USA-states', 
                    hover_name="state",
                    range_color=[25,75],
                    color_continuous_scale = 'RdBu',
                    scope="usa",
                    title='2020 Votes By Party')

# fig.write_image("vote_party.png")

fig.show()

![Figure](map_png/vote_party.png)

# Votes per Electoral Vote
This is calculated using the following equation:

**Votes per Electoral vote = (Total votes in state)/(Electoral votes per state)**

This should show how many individual votes are cast in each state for each electoral vote the state contributes. A lower value suggests each vote would theoretically have a higher influence in determining which candidate receives the state's electoral votes.

In [27]:
import warnings
import pandas as pd
import plotly.express as px

warnings.filterwarnings("ignore")
df_president_county = pd.read_csv('/Users/kellyg/maps-project/data/president_county_candidate.csv')
df_president_county = df_president_county[df_president_county.party.isin(['DEM', 'REP'])]
df_president_county = df_president_county.groupby(['state', 'party'])["total_votes"].sum().reset_index()
state_codes = pd.read_csv('/Users/kellyg/maps-project/data/world_country_and_usa_states_latitude_and_longitude_values.csv')
state_codes = state_codes[['usa_state', 'usa_state_code']]
df_president_county = df_president_county.merge(state_codes, left_on='state', right_on='usa_state')
df_president_county = df_president_county.drop(['usa_state'], axis=1)

# dep and rep columns
df_president_county_dummy = pd.get_dummies(df_president_county['party'])
df_president_county = df_president_county.join(df_president_county_dummy)

# votes by party
df_president_county['DEM_votes'] = df_president_county['DEM'] * df_president_county['total_votes']
df_president_county['REP_votes'] = df_president_county['REP'] * df_president_county['total_votes']
df_president_county = df_president_county.groupby(['state', 'usa_state_code'])[["DEM_votes", "REP_votes"]].sum().reset_index()

# electoral votes
df_electoral_votes = pd.read_csv('/Users/kellyg/maps-project/data/electoral_college.csv')
df_electoral_votes.columns = df_electoral_votes.columns.str.strip()
df_electoral_votes.rename(columns={'State': 'state'}, inplace=True)
df_president_county = pd.merge(df_president_county, df_electoral_votes, on='state', how='left')

# total votes per state and dc electoral votes = 3
df_president_county['total_votes'] = df_president_county['DEM_votes'] + df_president_county['REP_votes']
df_president_county['Electoral Votes'] = df_president_county.apply(
    lambda row: 3 if pd.isna(row['Electoral Votes']) and row['state'] == 'District of Columbia' else row['Electoral Votes'],
    axis=1
)

# votes per electoral vote
df_president_county['votes_per_electoral_vote'] = df_president_county['total_votes'] / df_president_county['Electoral Votes']

# map
fig = px.choropleth(
    df_president_county, 
    locations="usa_state_code", 
    color="votes_per_electoral_vote", 
    locationmode='USA-states', 
    hover_name="state",
    range_color=[df_president_county['votes_per_electoral_vote'].min(), df_president_county['votes_per_electoral_vote'].max()],
    color_continuous_scale='Viridis',  
    scope="usa",
    title='Votes per Electoral Vote by State'
)
# fig.write_image("vote_per_electoral_vote.png")
fig.show()


![Figure](map_png/vote_per_electoral_vote.png)

# Votes Clout Index
This is calculated using the following equation:

**Votes per Electoral vote = (State electoral votes per vote)/(US electoral votes per vote)**

In both the numerator and denominator, divide the electoral votes by the total votes -- in the numerator, focus on only the individual state, and in the denominator, calculate total electoral votes over total national votes. Each state should have the same denominator, but this metric should make the data more understandable. 

This should show how much influence an individual vote has in each state compared to other states. For example, a state like Wyoming is influenced more by individual votes than other states, since there is lower voter turnout.

In [24]:
import warnings
import pandas as pd
import plotly.express as px

warnings.filterwarnings("ignore")

df_president_county = pd.read_csv('/Users/kellyg/maps-project/data/president_county_candidate.csv')
df_president_county = df_president_county[df_president_county.party.isin(['DEM', 'REP'])]
df_president_county = df_president_county.groupby(['state', 'party'])["total_votes"].sum().reset_index()
state_codes = pd.read_csv('/Users/kellyg/maps-project/data/world_country_and_usa_states_latitude_and_longitude_values.csv')
state_codes = state_codes[['usa_state', 'usa_state_code']]
df_president_county = df_president_county.merge(state_codes, left_on='state', right_on='usa_state')
df_president_county = df_president_county.drop(['usa_state'], axis=1)

# dem and rep columns
df_president_county_dummy = pd.get_dummies(df_president_county['party'])
df_president_county = df_president_county.join(df_president_county_dummy)

# votes by party
df_president_county['DEM_votes'] = df_president_county['DEM'] * df_president_county['total_votes']
df_president_county['REP_votes'] = df_president_county['REP'] * df_president_county['total_votes']
df_president_county = df_president_county.groupby(['state', 'usa_state_code'])[["DEM_votes", "REP_votes"]].sum().reset_index()

# electoral votes 
df_electoral_votes = pd.read_csv('/Users/kellyg/maps-project/data/electoral_college.csv')
df_electoral_votes.columns = df_electoral_votes.columns.str.strip()
df_electoral_votes.rename(columns={'State': 'state'}, inplace=True)
df_president_county = pd.merge(df_president_county, df_electoral_votes, on='state', how='left')

#  total votes per state
df_president_county['total_votes'] = df_president_county['DEM_votes'] + df_president_county['REP_votes']
df_president_county['Electoral Votes'] = df_president_county.apply(
    lambda row: 3 if pd.isna(row['Electoral Votes']) and row['state'] == 'District of Columbia' else row['Electoral Votes'],
    axis=1
)

# total US votes and total US electoral votes 
total_us_votes = df_president_county['total_votes'].sum()
total_us_electoral_votes = df_president_county['Electoral Votes'].sum()

# VOTE CLOUT INDEX
df_president_county['vote_clout_index'] = (
    (df_president_county['Electoral Votes'] / df_president_county['total_votes']) / 
    (total_us_electoral_votes / total_us_votes)
)

# map
fig = px.choropleth(
    df_president_county, 
    locations="usa_state_code", 
    color="vote_clout_index", 
    locationmode='USA-states', 
    hover_name="state",
    range_color=[df_president_county['vote_clout_index'].min(), df_president_county['vote_clout_index'].max()],
    color_continuous_scale='Viridis',  
    scope="usa",
    title='Vote Clout Index by State'
)
# fig.write_image("vote_clout_index.png")
fig.show()


![Figure](map_png/vote_clout_index.png)

# Margin Sensitivity Score
This is calculated using the following equation:

**Votes per Electoral vote = (|Victory Margin|)/(Total votes)**

This value shows the percentage of votes needed in each state to change the majority party in the state. A higher value would indicate a higher percentage of votes would need to have been cast for the opposite party to change the outcome, indicating the state is unlikely to flip and therefore individual votes have a lower impact

In [25]:
import warnings
import pandas as pd
import plotly.express as px

warnings.filterwarnings("ignore")

df_president_county = pd.read_csv('/Users/kellyg/maps-project/data/president_county_candidate.csv')
df_president_county = df_president_county[df_president_county.party.isin(['DEM', 'REP'])]
df_president_county = df_president_county.groupby(['state', 'party'])["total_votes"].sum().reset_index()
state_codes = pd.read_csv('/Users/kellyg/maps-project/data/world_country_and_usa_states_latitude_and_longitude_values.csv')
state_codes = state_codes[['usa_state', 'usa_state_code']]
df_president_county = df_president_county.merge(state_codes, left_on='state', right_on='usa_state')
df_president_county = df_president_county.drop(['usa_state'], axis=1)

# dem and rep
df_president_county_dummy = pd.get_dummies(df_president_county['party'])
df_president_county = df_president_county.join(df_president_county_dummy)

#  votes by party
df_president_county['DEM_votes'] = df_president_county['DEM'] * df_president_county['total_votes']
df_president_county['REP_votes'] = df_president_county['REP'] * df_president_county['total_votes']
df_president_county = df_president_county.groupby(['state', 'usa_state_code'])[["DEM_votes", "REP_votes"]].sum().reset_index()

df_electoral_votes = pd.read_csv('/Users/kellyg/maps-project/data/electoral_college.csv')
df_electoral_votes.columns = df_electoral_votes.columns.str.strip()
df_electoral_votes.rename(columns={'State': 'state'}, inplace=True)
df_president_county = pd.merge(df_president_county, df_electoral_votes, on='state', how='left')

#  total votes per state 
df_president_county['total_votes'] = df_president_county['DEM_votes'] + df_president_county['REP_votes']
df_president_county['Electoral Votes'] = df_president_county.apply(
    lambda row: 3 if pd.isna(row['Electoral Votes']) and row['state'] == 'District of Columbia' else row['Electoral Votes'],
    axis=1
)

# margin of victory
df_president_county['margin_of_victory'] = df_president_county['DEM_votes'] - df_president_county['REP_votes']

# MARGIN SENSITIVITY SCORE
df_president_county['margin_sensitivity_score'] = (
    df_president_county['margin_of_victory'].abs() / df_president_county['total_votes']) * 100

#map
fig = px.choropleth(
    df_president_county, 
    locations="usa_state_code", 
    color="margin_sensitivity_score", 
    locationmode='USA-states', 
    hover_name="state",
    range_color=[df_president_county['margin_sensitivity_score'].min(), 45],
    color_continuous_scale='Viridis',  
    scope="usa",
    title='Margin Sensitivity Score by State'
)
# fig.write_image("margin_sensitivity.png")
fig.show()


![Figure](map_png/margin_sensitivity.png)

# Effective Voter Power
This is calculated using the following equation:

**Votes per Electoral vote = (Electoral Votes)/(|Victory Margin|)**

Similar to the margin sensitivity score, this calculates based on the relationship between the victory margin and number of votes (though this uses electoral votes instead of total votes). If a state has a higher effective voter power, this suggests the state most likely has a lower victory margin (or it was a "tighter" race in that state). This is clear on the map, since GA, NC, PA, WI, AZ, and NV are visibly highest in effective voter power, and these are historically swing states.

In [26]:
import warnings
import pandas as pd
import plotly.express as px

warnings.filterwarnings("ignore")
df_president_county = pd.read_csv('/Users/kellyg/maps-project/data/president_county_candidate.csv')
df_president_county = df_president_county[df_president_county.party.isin(['DEM', 'REP'])]
df_president_county = df_president_county.groupby(['state', 'party'])["total_votes"].sum().reset_index()
state_codes = pd.read_csv('/Users/kellyg/maps-project/data/world_country_and_usa_states_latitude_and_longitude_values.csv')
state_codes = state_codes[['usa_state', 'usa_state_code']]
df_president_county = df_president_county.merge(state_codes, left_on='state', right_on='usa_state')
df_president_county = df_president_county.drop(['usa_state'], axis=1)

# dep and rep columns
df_president_county_dummy = pd.get_dummies(df_president_county['party'])
df_president_county = df_president_county.join(df_president_county_dummy)

# votes by party
df_president_county['DEM_votes'] = df_president_county['DEM'] * df_president_county['total_votes']
df_president_county['REP_votes'] = df_president_county['REP'] * df_president_county['total_votes']
df_president_county = df_president_county.groupby(['state', 'usa_state_code'])[["DEM_votes", "REP_votes"]].sum().reset_index()

# electoral votes
df_electoral_votes = pd.read_csv('/Users/kellyg/maps-project/data/electoral_college.csv')
df_electoral_votes.columns = df_electoral_votes.columns.str.strip()
df_electoral_votes.rename(columns={'State': 'state'}, inplace=True)
df_president_county = pd.merge(df_president_county, df_electoral_votes, on='state', how='left')

# Calculate the margin of victory as the absolute difference between DEM and REP votes
df_president_county['margin_of_victory'] = abs(df_president_county['DEM_votes'] - df_president_county['REP_votes'])

# Apply the formula for effective voter power
df_president_county['effective_voter_power'] = (df_president_county['Electoral Votes'] / df_president_county['margin_of_victory'])*1000

# Handle cases where margin_of_victory is zero to avoid division by zero
# df_president_county['effective_voter_power'] = df_president_county['effective_voter_power'].replace([float('inf'), -float('inf')], 0)

# Map visualization
fig = px.choropleth(
    df_president_county, 
    locations="usa_state_code", 
    color="effective_voter_power", 
    locationmode='USA-states', 
    hover_name="state",
    range_color=[df_president_county['effective_voter_power'].min(), df_president_county['effective_voter_power'].max()],
    color_continuous_scale='Viridis',  
    scope="usa",
    title='Effective Voter Power by State'
)
# fig.write_image("effective_voter_power.png")
fig.show()


![Figure](map_png/effective_voter_power.png)